In [ ]:
mport numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
s_data = yf.download('AAPL', start='2020-01-01', end='2022-01-01')
data = s_data['Close'].values
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data.reshape(-1, 1))
seq_leng = 20
X, y = [], []
for i in range(len(data) - seq_leng):
    X.append(data[i:i+seq_leng])
    y.append(data[i+seq_leng])
X, y = np.array(X), np.array(y)
split_ratio = 0.8
split_index = int(split_ratio * len(data))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=64)
predicted_stock_prices = model.predict(X_test)
predicted_stock_prices = scaler.inverse_transform(predicted_stock_prices)
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(s_data.index[split_index+seq_leng:], s_data['Close'][split_index+seq_leng:], label='Actual Stock Prices', color='b')
plt.plot(s_data.index[split_index+seq_leng:], predicted_stock_prices, label='Predicted Stock Prices', color='r')
plt.legend()
plt.title('Stock Price Prediction for AAPL')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.show()